In [44]:
import numpy as np
import gzip
f = gzip.open('./eLIDAR/P3/P3.extendedFrags.fastq.gz')
doc1 = f.readlines()
f.close()
# 6 4 1

In [45]:
total_reads = len(doc1)/4
print('total # of reads:',total_reads)

total # of reads: 334022.0


In [46]:
def string2ascii(s, l):
    scores = [ord(c)-33 for c in s]
    for i in range(len(scores)-3):
        if ((scores[i]+scores[i+1]+scores[i+2]+scores[i+3])/3)<27:
            return 1
    return sum(scores)/l

In [47]:
seq_only=[]
seq_l=[]
smaller=0
for i in range(0, len(doc1[:]), 4):
    seq1 = doc1[i+1]
    seq1 = str(seq1[:-1], encoding='utf-8')
    score1 = doc1[i+3]
    score1 = str(score1[:-1], encoding='utf-8')
    l = len(seq1)
    if l < 272:
        smaller+=1
        continue
    if string2ascii(score1,l) > 30: # 1 in 1000
        seq_only.append(seq1)
        seq_l.append(l)
print('High quality reads:',len(seq_l))
print("--> Fraction of high quality reads:", len(seq_l)/total_reads)
print("Reads not long enough:", smaller)

High quality reads: 304176
--> Fraction of high quality reads: 0.9106466041158966
Reads not long enough: 7114


In [48]:
np.median(seq_l)

356.0

In [49]:
np.std(seq_l)

6.869340967595127

In [50]:
c = 0
ed = 0
non = 0
non_seq = []

for s1 in seq_only[:]:
  if   'CCTCCGTTTAGGTGGGTGG' in str(s1):
    c=c+1
  elif 'CCTCCGTTTGGGTGGGTGG' in str(s1):
    ed=ed+1
  else:
    non=non+1
    non_seq.append(s1)

print('Not edited:',c)
print('Edited:',ed)
print('Non-amplicon reads:',non)
print('Total reads:',c+ed+non, end='\n\n')

print('% of non-amplicon reads:', 100*non/(c+ed+non))
print('% of edited reporter (all reads)', 100*ed/(c+ed+non))
print('% of not edited reporter (all reads)', 100*c/(c+ed+non))
print('----> % of edited reporter (amplicon reads)', 100*ed/(c+ed))
print('% of not edited reporter (amplicon reads)', 100*c/(c+ed))

Not edited: 298146
Edited: 698
Non-amplicon reads: 5332
Total reads: 304176

% of non-amplicon reads: 1.7529325127557729
% of edited reporter (all reads) 0.22947241070958918
% of not edited reporter (all reads) 98.01759507653463
----> % of edited reporter (amplicon reads) 0.2335666769284309
% of not edited reporter (amplicon reads) 99.76643332307157


In [51]:
#eLIDAR
#N1 = 3.3681
#n2 = 3.5284
#n3 = 3.3679
#p1 = 0.2618
#p2 = 0.2745
#p3 = 0.2335

In [52]:
len(non_seq)

5332

In [53]:
from Bio import pairwise2

In [54]:
e=0
ne=0
for s in non_seq[:]:
    alignment = pairwise2.align.localms(s, "CCGTTTAGGTGGG", 1, 0, -100, -100)
    score = alignment[0].score
    start=alignment[0].start
    if score < 12:
        alignment = pairwise2.align.localms(s, "CCGTTTGGGTGGG", 1, 0, -100, -100)
        score = alignment[0].score
        if score < 12 or start != alignment[0].start:
            continue
        else:
            e=e+1
    else:
        ne=ne+1

In [55]:
e, ne, e/ne

(4, 2075, 0.0019277108433734939)

In [56]:
c+=ne
ed+=e

print('Not edited:',c)
print('Edited:',ed)
print('Final fraction:',100*ed/(ed+c))

Not edited: 300221
Edited: 702
Final fraction: 0.23328226822143872
